In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/1.url'):
    shutil.rmtree('./file/1.url')

os.makedirs('./file/1.url')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        url = input_.loc[a, 'Url']
        
        dict_param = {}
        list_param = url.split('?')[1].split('&') if '?' in url else []
        for param in list_param:
            dict_param[param.split('=')[0]] = param.split('=')[1]
        
        dict_param.pop('page', '')
        dict_param['sort'] = 'alphaasc'
        dict_param['limit'] = '96'
        
        url = url.split('?')[0] + '?'
        for key, value in dict_param.items():
            url += f'{key}={value}&'

        try:
            list_url, i = [], 1
            while True:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(f'{url}page={i}',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
    
                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
    
                list_url += html.xpath('//h4[@class="card-title NoMargin"]/a/@href')

                # = = = = = = = = = = = = = = =
    
                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()
    
                print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']} > Page {i}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')
    
                # = = = = = = = = = = = = = = =
    
                i += 1
    
                # = = = = = = = = = = = = = = =
    
                if not html.xpath('//li[@class="pagination-item pagination-item--next"]'):
                    break
                    
            # = = = = = = = = = = = = = = =
    
            if not list_url:
                raise
        
            # = = = = = = = = = = = = = = =

            output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                                   'Url': list_url})
            
            output = output.sort_values(by=['No'],
                                        ascending=[True],
                                        ignore_index=True)
            
            output.to_excel(f'''./file/1.url/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/url_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/1.url/{file}',
              f'''./file/1.url/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[尝试次数：1] - 1.https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?sort=alphaasc&limit=96&page=1 > Page 1
[剩余数量：0] - [当前时间：10:10:25]

[尝试次数：1] - 2.https://www.cardone.com/drivetrain/driveline-axles/driveshaft-prop-shaft/?limit=96&sort=alphaasc > Page 1
[剩余数量：0] - [当前时间：10:10:26]

[尝试次数：1] - 1.https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?sort=alphaasc&limit=96&page=1 > Page 2
[剩余数量：0] - [当前时间：10:10:28]

[尝试次数：1] - 2.https://www.cardone.com/drivetrain/driveline-axles/driveshaft-prop-shaft/?limit=96&sort=alphaasc > Page 2
[剩余数量：0] - [当前时间：10:10:32]

[尝试次数：1] - 2.https://www.cardone.com/drivetrain/driveline-axles/driveshaft-prop-shaft/?limit=96&sort=alphaasc > Page 3
[剩余数量：0] - [当前时间：10:10:34]

[尝试次数：1] - 2.https://www.cardone.com/drivetrain/driveline-axles/driveshaft-prop-shaft/?limit=96&sort=alphaasc > Page 4
[剩余数量：0] - [当前时间：10:10:38]

[尝试次数：1] - 2.https://www.cardone.com/drivetrain/driveline-axles/driveshaft-prop-shaft/?limit=96&sort=

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 2758.50it/s]

Done ~
